It happens all the time: someone gives you data containing malformed strings,
Python, lists and missing data. How do you tidy it up so you can get on with the
analysis?
Take this monstrosity as the DataFrame to use in the following puzzles:

In [1]:
import pandas as pd
import numpy as np

In [43]:
df = pd.DataFrame({'From_To': ['LoNDon_paris', 'MAdrid_miLAN',
'londON_StockhOlm',
'Budapest_PaRis', 'Brussels_londOn'],
'FlightNumber': [10045, np.nan, 10065, np.nan, 10085],
'RecentDelays': [[23, 47], [], [24, 43, 87], [13], [67, 32]],
'Airline': ['KLM(!)', '<Air France> (12)', '(British Airways. )',
'12. Air France', '"Swiss Air"']})

In [3]:
df.shape

(5, 4)

In [4]:
df.head(10)

,From_To,FlightNumber,RecentDelays,Airline
0,LoNDon_paris,10045.0,"[23, 47]",KLM(!)
1,MAdrid_miLAN,NaN,[],<Air France> (12)
2,londON_StockhOlm,10065.0,"[24, 43, 87]",(British Airways. )
3,Budapest_PaRis,NaN,[13],12. Air France
4,Brussels_londOn,10085.0,"[67, 32]","""Swiss Air"""


In [5]:
df.describe()

,FlightNumber
count,3.0
mean,10065.0
std,20.0
min,10045.0
25%,10055.0
50%,10065.0
75%,10075.0
max,10085.0


In [47]:
df.dtypes

From_To          object
FlightNumber    float64
RecentDelays     object
Airline          object
dtype: object

# 1. Some values in the the FlightNumber column are missing. These numbers are
meant to increase by 10 with each row so 10055 and 10075 need to be put in
place. Fill in these missing numbers and make the column an integer column
(instead of a float column).

In [51]:
df.loc[df["FlightNumber"].isna(),"FlightNumber"].fillna(value=10)

1    10.0
3    10.0
Name: FlightNumber, dtype: float64

In [42]:
df["FlightNumber"] = df["FlightNumber"].fillna(method="ffill").apply(lambda x)
df

,From_To,FlightNumber,RecentDelays,Airline
0,LoNDon_paris,10045.0,"[23, 47]",KLM(!)
1,MAdrid_miLAN,10045.0,[],<Air France> (12)
2,londON_StockhOlm,10065.0,"[24, 43, 87]",(British Airways. )
3,Budapest_PaRis,10065.0,[13],12. Air France
4,Brussels_londOn,10085.0,"[67, 32]","""Swiss Air"""


In [24]:
prev=df["FlightNumber"][0]
l=[]
for i in df["FlightNumber"]:
    if pd.isna(i):
        l.append(prev + 10)
    else:
        l.append(i)
    prev=i

In [26]:
l

[10045.0, 10055.0, 10065.0, 10075.0, 10085.0]

In [28]:
df["FlightNumber"] = l

In [29]:
df.head(5)

,From_To,FlightNumber,RecentDelays,Airline
0,LoNDon_paris,10045.0,"[23, 47]",KLM(!)
1,MAdrid_miLAN,10055.0,[],<Air France> (12)
2,londON_StockhOlm,10065.0,"[24, 43, 87]",(British Airways. )
3,Budapest_PaRis,10075.0,[13],12. Air France
4,Brussels_londOn,10085.0,"[67, 32]","""Swiss Air"""


Another way is 

In [28]:
df["FlightNumber"] = df["FlightNumber"].ffill() + df.groupby(df["FlightNumber"].notnull().cumsum()).cumcount()*10

In [39]:
df["FlightNumber"] = df["FlightNumber"].astype("int64")

In [40]:
df.head(10)

,From_To,FlightNumber,RecentDelays,Airline
0,LoNDon_paris,10045,"[23, 47]",KLM(!)
1,MAdrid_miLAN,10055,[],<Air France> (12)
2,londON_StockhOlm,10065,"[24, 43, 87]",(British Airways. )
3,Budapest_PaRis,10075,[13],12. Air France
4,Brussels_londOn,10085,"[67, 32]","""Swiss Air"""


In [41]:
df.dtypes

From_To         object
FlightNumber     int64
RecentDelays    object
Airline         object
dtype: object

In [23]:
#new_df = df["FlightNumber"].astype(int)

# 2. The From_To column would be better as two separate columns! Split each
string on the underscore delimiter _ to give a new temporary DataFrame with
the correct values. Assign the correct column names to this temporary
DataFrame.

In [51]:
df[["From","To"]] = df["From_To"].str.split("_",expand=True)

In [78]:
df

,FlightNumber,Airline,From,To,delay_1,delay_2,delay_3,delay_1,delay_2,delay_3
0,10045.0,KLM(!),London,Paris,23.0,47.0,NaN,23.0,47.0,NaN
1,NaN,<Air France> (12),Madrid,Milan,NaN,NaN,NaN,NaN,NaN,NaN
2,10065.0,(British Airways. ),London,Stockholm,24.0,43.0,87.0,24.0,43.0,87.0
3,NaN,12. Air France,Budapest,Paris,13.0,NaN,NaN,13.0,NaN,NaN
4,10085.0,"""Swiss Air""",Brussels,London,67.0,32.0,NaN,67.0,32.0,NaN


# 3. Notice how the capitalisation of the city names is all mixed up in this
temporary DataFrame. Standardise the strings so that only the first letter is
uppercase (e.g. "londON" should become "London".)

In [61]:
temp_df = df
temp_df["From"] = temp_df["From"].str.title()

In [62]:
temp_df["To"] = temp_df["To"].str.title()

In [63]:
temp_df.head()

,From_To,FlightNumber,RecentDelays,Airline,From,To
0,LoNDon_paris,10045.0,"[23, 47]",KLM(!),London,Paris
1,MAdrid_miLAN,NaN,[],<Air France> (12),Madrid,Milan
2,londON_StockhOlm,10065.0,"[24, 43, 87]",(British Airways. ),London,Stockholm
3,Budapest_PaRis,NaN,[13],12. Air France,Budapest,Paris
4,Brussels_londOn,10085.0,"[67, 32]","""Swiss Air""",Brussels,London


# 4. Delete the From_To column from df and attach the temporary DataFrame
from the previous questions.

In [65]:
df = temp_df.drop("From_To",axis=1)

In [66]:
df.head(5)

,FlightNumber,RecentDelays,Airline,From,To
0,10045.0,"[23, 47]",KLM(!),London,Paris
1,NaN,[],<Air France> (12),Madrid,Milan
2,10065.0,"[24, 43, 87]",(British Airways. ),London,Stockholm
3,NaN,[13],12. Air France,Budapest,Paris
4,10085.0,"[67, 32]","""Swiss Air""",Brussels,London


# 5. In the RecentDelays column, the values have been entered into the
DataFrame as a list. We would like each first value in its own column, each second value in its own column, and so on. If there isn't an Nth value, the value
should be NaN.
Expand the Series of lists into a DataFrame named delays, rename the columns
delay_1, delay_2, etc. and replace the unwanted RecentDelays column in df
with delays.

In [67]:
type(df["RecentDelays"])

pandas.core.series.Series

In [72]:
delays = pd.DataFrame(df["RecentDelays"].values.tolist(),columns=["delay_1","delay_2","delay_3"])

In [73]:
delays

,delay_1,delay_2,delay_3
0,23.0,47.0,NaN
1,NaN,NaN,NaN
2,24.0,43.0,87.0
3,13.0,NaN,NaN
4,67.0,32.0,NaN


In [75]:
df=pd.concat([df, delays], axis=1)
df

,FlightNumber,RecentDelays,Airline,From,To,delay_1,delay_2,delay_3,delay_1,delay_2,delay_3
0,10045.0,"[23, 47]",KLM(!),London,Paris,23.0,47.0,NaN,23.0,47.0,NaN
1,NaN,[],<Air France> (12),Madrid,Milan,NaN,NaN,NaN,NaN,NaN,NaN
2,10065.0,"[24, 43, 87]",(British Airways. ),London,Stockholm,24.0,43.0,87.0,24.0,43.0,87.0
3,NaN,[13],12. Air France,Budapest,Paris,13.0,NaN,NaN,13.0,NaN,NaN
4,10085.0,"[67, 32]","""Swiss Air""",Brussels,London,67.0,32.0,NaN,67.0,32.0,NaN


In [77]:
df.drop("RecentDelays",inplace=True,axis=1)
df

,FlightNumber,Airline,From,To,delay_1,delay_2,delay_3,delay_1,delay_2,delay_3
0,10045.0,KLM(!),London,Paris,23.0,47.0,NaN,23.0,47.0,NaN
1,NaN,<Air France> (12),Madrid,Milan,NaN,NaN,NaN,NaN,NaN,NaN
2,10065.0,(British Airways. ),London,Stockholm,24.0,43.0,87.0,24.0,43.0,87.0
3,NaN,12. Air France,Budapest,Paris,13.0,NaN,NaN,13.0,NaN,NaN
4,10085.0,"""Swiss Air""",Brussels,London,67.0,32.0,NaN,67.0,32.0,NaN
